# ToDo

## Path generation API
### Single Robot Path Generation
- Generate coverage path from given area map and list of relevant points (start, end, fuel).
- Generate fuel path from given fuel map and coverage path
- Generate combined path 

### Multiple Robot Path Generation
- Obtain area division map from DARP
- Pass area division map to all algos with relevant cell value to represent mapping area.
- Same interface as single robot path generation.

## Testing
### Functions to visualize


### Metrics
#### Percents
- **Coverage** - Percentage of non obstacle cells covered by the path.
- **Redundancy** - Percentage of cells from the path that have been revisited (backtracked).

#### Timings
- **Coverage Path computation time** - Time taken by the algorithm to compute the coverage path.
- **Fuel Path computation time** - Time taken to compute the refuel path.
- **Fuel dist map generation time** - Time required to generate the fuel distance map.

#### Numbers
- **Coverage cells** - Number of cells to be covered.
- **Non covered cells** - Cells that haven't been covered.
- **Backtrack** - Count of coverage path cells that have been revisited.
- **Coverage Path length** - Number of steps in the area coverage path.
- **Fuel Path length** - Number of steps in the refueling path.
- **Total Path length** - Sum of the above two points.
    
### Running Tests On
- Resolutions, Get obstacle maps of the following resolutions.
    - (32, 32)
    - (50, 50)
    - (144, 50)
    - (256, 144)
    - (256, 256)
- Obstacle types
    - None
    - Point obstacles (single cell obstacles generated randomly).
    - Region obstacles (region obstacles that have been painted).
        - Corners
        - Central
        - Caves
        - Pipes, Bottlenecks
    - Combination of both.

# Testing

In [44]:
# Import

import numpy as np
import matplotlib.pyplot as plt

from PIL import Image
from pathlib import Path

In [92]:
# Constants

OB = -1 # Value for obstacle.
NO = 0  # Value for region to be mapped.
AMS_PATH = "./test_maps/" # Area Maps Paths

## Getting area maps

In [93]:
resolutions = [(32, 32),(50, 50),(144, 50),(256, 144),(256,256)]

def imshow(area_map,figsize=(5,5),cmap="viridis"):
    """
    Display with no interpolation.
    """
    plt.figure(figsize=figsize)
    ax = plt.imshow(area_map,interpolation='none',cmap=cmap)
    plt.axis('off');
    return ax

def generate_no_obs_area_map(resolutions=resolutions):
    """
    resolutions : list of tuples [(rows, cols)]
    """
    area_maps = []
    for res in resolutions:
        area_maps.append(np.zeros(res))
    return area_maps

def get_area_map(path):
    """
    path : path to area map png, png should have only 
    0 255 as values.
    returns area_map with cell values
    obstacle : OBS
    non obstacle : NOB
    """
    am = np.array(Image.open(path))
    ma = np.array(am).mean(axis=2) == 255
    am = np.int8(np.zeros(ma.shape))
    am[ma] = NO
    am[~ma]  = OB
    return am

def get_area_map_list(folder_path=AMS_PATH):
    """
    folder_path : path to the folder contiaining the maps (.png)
    """
    ams = []
    for path in Path(folder_path).iterdir():
        try:
            ams.append(get_area_map(path))
        except:
            continue
    return ams